In [19]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from tqdm import tqdm
import urllib.parse
import pandas as pd
import re

In [20]:
def get_long_lat(location, kabupaten):
    import time
    # Gabungkan location dan kabupaten untuk query lebih presisi
    search_query = f"{location} {kabupaten}"
    encoded_location = urllib.parse.quote(search_query)

    driver.get("https://www.google.com/maps/search/{}".format(encoded_location))
    driver.implicitly_wait(3)
    
    # Get the initial page source
    initial_page_source = driver.page_source
    
    # Wait for the page source to change with a timeout of 10 seconds
    wait = WebDriverWait(driver, 10)
    wait.until(lambda driver: driver.page_source != initial_page_source)

    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    # CASE 1: Ditemukan list banyak hasil pencarian
    if soup.find('h1', class_="fontTitleLarge IFMGgb"):
        print("Case 1: Multiple results ditemukan")
        
        # Ambil link dari hasil pencarian pertama
        link_element = soup.find('a', class_='hfpxzc')
        if link_element:
            url = link_element.get('href')
            print(f"Link: {url}")
            driver.get(url)  # Buka link
            
            # Tunggu page source berubah setelah klik
            initial_page_source = driver.page_source
            wait = WebDriverWait(driver, 10)
            wait.until(lambda driver: driver.page_source != initial_page_source)
            
            # Tunggu URL berubah menjadi format yang stabil (dengan /@lat,lng)
            for attempt in range(10):
                current_url = driver.current_url
                if '/@' in current_url and re.search(r'/@-?\d+\.\d+,-?\d+\.\d+', current_url):
                    print(f"URL stabil ditemukan pada attempt {attempt}")
                    break
                if attempt < 9:
                    time.sleep(1)
            
            # Ambil soup dari halaman setelah klik
            html_after_click = driver.page_source
            soup_after_click = BeautifulSoup(html_after_click, "html.parser")
            
            # Ambil nama Google dari h1 dengan class "DUwDvf lfPIob"
            h1_element = soup_after_click.find('h1', class_='DUwDvf lfPIob')
            if h1_element:
                nama_google = h1_element.get_text(strip=True)
                print(f"Nama Google: {nama_google}")
            else:
                nama_google = None
                print("Nama Google tidak ditemukan")
            
            # Extract koordinat dari URL
            current_url = driver.current_url
            print(f"Final URL: {current_url}")
            
            lat_lng_match = re.search(r'/@(-?\d+\.\d+),(-?\d+\.\d+)', current_url)
            if lat_lng_match:
                latitude = float(lat_lng_match.group(1))
                longitude = float(lat_lng_match.group(2))
                print(f"Latitude: {latitude}, Longitude: {longitude}")
                return [latitude, longitude, nama_google, current_url, 0]
            else:
                print("Koordinat tidak ditemukan di URL")
                return [None, None, nama_google, current_url, 1]
        else:
            print("Link tidak ditemukan")
            return [None, None, None, None, 1]
    
    # CASE 2: Ditemukan tepat 1 hasil pencarian langsung
    elif soup.find('h1', class_='DUwDvf lfPIob'):
        print("Case 2: 1 hasil ditemukan langsung")
        
        # Tunggu URL berubah menjadi format yang stabil (dengan /@lat,lng)
        for attempt in range(10):
            current_url = driver.current_url
            if '/@' in current_url and re.search(r'/@-?\d+\.\d+,-?\d+\.\d+', current_url):
                print(f"URL stabil ditemukan pada attempt {attempt}")
                break
            if attempt < 9:
                time.sleep(1)
        
        # Ambil nama Google
        h1_element = soup.find('h1', class_='DUwDvf lfPIob')
        if h1_element:
            nama_google = h1_element.get_text(strip=True)
            print(f"Nama Google: {nama_google}")
        else:
            nama_google = None
            print("Nama Google tidak ditemukan")
        
        # Extract koordinat dari URL
        current_url = driver.current_url
        print(f"Final URL: {current_url}")
        
        lat_lng_match = re.search(r'/@(-?\d+\.\d+),(-?\d+\.\d+)', current_url)
        if lat_lng_match:
            latitude = float(lat_lng_match.group(1))
            longitude = float(lat_lng_match.group(2))
            print(f"Latitude: {latitude}, Longitude: {longitude}")
            return [latitude, longitude, nama_google, current_url, 0]
        else:
            print("Koordinat tidak ditemukan di URL")
            return [None, None, nama_google, current_url, 1]
    
    # CASE 3: Tidak ditemukan
    else:
        print("Case 3: Lokasi tidak ditemukan")
        return [None, None, None, None, 2]

In [21]:
file_input = 'Flat_sekolah_terdampak.csv'

df = pd.read_csv('input/{}'.format(file_input), sep=';')

if 'latitude' not in df.columns:
    df['latitude'] = ''

if 'longitude' not in df.columns:
    df['longitude'] = ''

if 'nama_google' not in df.columns:
    df['nama_google'] = ''

if 'url' not in df.columns:
    df['url'] = ''

In [22]:
df_na = df.loc[df['latitude'].isna() | (df['latitude'] == '')]
df_no_na = df.loc[~(df['latitude'].isna() | (df['latitude'] == ''))]

# Untuk df_na: set isna=1 kecuali sudah 2 (tidak ditemukan sebelumnya)
df_na['isna'] = df_na['isna'].apply(lambda x: 1 if (pd.isna(x) or x != 2) else 2)
# Untuk df_no_na: set isna=0 (ditemukan)
df_no_na['isna'] = 0

C:\Users\fawcet.makay\AppData\Local\Temp\ipykernel_23796\1227467311.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_na['isna'] = df_na['isna'].apply(lambda x: 1 if (pd.isna(x) or x != 2) else 2)
C:\Users\fawcet.makay\AppData\Local\Temp\ipykernel_23796\1227467311.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_no_na['isna'] = 0


In [23]:
df_na.head(5)

,npsn_isian,nama_isian,Jenjang,alamat_jalan,nama_dusun,desa_kelurahan,kecamatan,kabupaten,provinsi,lintang,bujur,latitude,longitude,nama_google,url,isna
0,69775026,AL0RIFDA,PAUD Sederajat,"PERUMNAS BRANDAN PERMAI, TANGKAHAN DURIAN",-,Tangkahan Durian,Kec. Berandan Barat,Kab. Langkat,Prov. Sumatera Utara,0.000000000000,0.000000000000,NaN,NaN,NaN,NaN,1
1,69790018,IKRA,PAUD Sederajat,BATANG DURIAN NO 1,-,PEKAN TANJUNG PURA,Kec. Tanjung Pura,Kab. Langkat,Prov. Sumatera Utara,-5.443.757.000.000,114.421.127.000.000,NaN,NaN,NaN,NaN,1
4,69932645,KB AL-BARRU,PAUD Sederajat,Gampong Lhok Merbo Kecamatan Sawang Kabupaten ...,Lhok Merbo,Lhok Meureubo,Kec. Sawang,Kab. Aceh Utara,Prov. Aceh,0.000000000000,0.000000000000,NaN,NaN,NaN,NaN,1
5,69992786,KB AL-HIJRAH,PAUD Sederajat,Gampong Lhok Reudeup Kecamatan Tanah Jambo Aye...,-,Lhok Reudeup,Kec. Tanah Jambo Aye,Kab. Aceh Utara,Prov. Aceh,5.081.100.000.000,97.459.900.000.000,NaN,NaN,NaN,NaN,1
7,69967119,KB AR-RIDHA,PAUD Sederajat,Gampong Cot Ara Kecamatan Baktiya Kabupaten Ac...,Cot Ara,Cot Ara,Kec. Baktiya,Kab. Aceh Utara,Prov. Aceh,0.000000000000,0.000000000000,NaN,NaN,NaN,NaN,1


In [24]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
options.add_argument('--disable-extensions')
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])

driver = webdriver.Chrome(options=options)

for i, r in tqdm(df_na.iterrows(), total=df_na.shape[0]):
    if r['isna'] == 1 :
        location_name = r['nama_isian']
        kabupaten = r['kabupaten']  # Ambil kabupaten dari dataframe
        df_na.loc[i, ['latitude', 'longitude', 'nama_google', 'url', 'isna']] = get_long_lat(location_name, kabupaten)

  0%|          | 1/4660 [00:01<2:22:22,  1.83s/it]

Case 3: Lokasi tidak ditemukan


  0%|          | 2/4660 [00:02<1:41:27,  1.31s/it]

Case 3: Lokasi tidak ditemukan


  0%|          | 3/4660 [00:03<1:24:03,  1.08s/it]

...

URL stabil ditemukan pada attempt 3
Nama Google: Paud KB Melati Putih
Final URL: https://www.google.com/maps/place/Paud+KB+Melati+Putih/@2.7809315,97.9483898,17z/data=!3m1!4b1!4m6!3m5!1s0x303073d2bc65f781:0x8cd854289bd21f36!8m2!3d2.7809315!4d97.9483898!16s%2Fg%2F11l73h4sdl?entry=ttu&g_ep=EgoyMDI2MDIwOC4wIKXMDSoASAFQAw%3D%3D
Latitude: 2.7809315, Longitude: 97.9483898
Case 1: Multiple results ditemukan
Link: https://www.google.com/maps/place/Baitul+Mal+Aceh+Tamiang/data=!4m7!3m6!1s0x303775e526041369:0xeaaa934881ea0912!8m2!3d4.3436174!4d98.0721523!16s%2Fg%2F11g3y7752b!19sChIJaRMEJuV1NzAREgnqgUiTquo?authuser=0&hl=en&rclk=1


  2%|▏         | 79/4660 [01:47<4:08:22,  3.25s/it]

URL stabil ditemukan pada attempt 3
Nama Google: Baitul Mal Aceh Tamiang
Final URL: https://www.google.com/maps/place/Baitul+Mal+Aceh+Tamiang/@4.3436174,98.0721523,17z/data=!3m1!4b1!4m6!3m5!1s0x303775e526041369:0xeaaa934881ea0912!8m2!3d4.3436174!4d98.0721523!16s%2Fg%2F11g3y7752b?authuser=0&hl=en&entry=ttu&g_ep=EgoyMDI2MDIwOC4wIKXMDSoASAFQAw%3D%3D
Latitude: 4.3436174, Longitude: 98.0721523
Case 2: 1 hasil ditemukan langsung


  2%|▏         | 79/4660 [01:49<1:45:56,  1.39s/it]


KeyboardInterrupt: 

In [25]:
df_na.head()

,npsn_isian,nama_isian,Jenjang,alamat_jalan,nama_dusun,desa_kelurahan,kecamatan,kabupaten,provinsi,lintang,bujur,latitude,longitude,nama_google,url,isna
0,69775026,AL0RIFDA,PAUD Sederajat,"PERUMNAS BRANDAN PERMAI, TANGKAHAN DURIAN",-,Tangkahan Durian,Kec. Berandan Barat,Kab. Langkat,Prov. Sumatera Utara,0.000000000000,0.000000000000,NaN,NaN,None,None,2
1,69790018,IKRA,PAUD Sederajat,BATANG DURIAN NO 1,-,PEKAN TANJUNG PURA,Kec. Tanjung Pura,Kab. Langkat,Prov. Sumatera Utara,-5.443.757.000.000,114.421.127.000.000,NaN,NaN,None,None,2
4,69932645,KB AL-BARRU,PAUD Sederajat,Gampong Lhok Merbo Kecamatan Sawang Kabupaten ...,Lhok Merbo,Lhok Meureubo,Kec. Sawang,Kab. Aceh Utara,Prov. Aceh,0.000000000000,0.000000000000,NaN,NaN,None,None,2
5,69992786,KB AL-HIJRAH,PAUD Sederajat,Gampong Lhok Reudeup Kecamatan Tanah Jambo Aye...,-,Lhok Reudeup,Kec. Tanah Jambo Aye,Kab. Aceh Utara,Prov. Aceh,5.081.100.000.000,97.459.900.000.000,-6.872247,107.522857,RA dan MI Nur Al - Hijrah,https://www.google.com/maps/place/RA+dan+MI+Nu...,0
7,69967119,KB AR-RIDHA,PAUD Sederajat,Gampong Cot Ara Kecamatan Baktiya Kabupaten Ac...,Cot Ara,Cot Ara,Kec. Baktiya,Kab. Aceh Utara,Prov. Aceh,0.000000000000,0.000000000000,NaN,NaN,None,None,2


In [26]:
# Gabungkan data yang sudah diproses
x = pd.concat([df_na, df_no_na])
x = x.sort_index()

# Hapus kolom 'isna' sebelum menyimpan (opsional, jika ingin)
# x = x.drop('isna', axis=1)

# Simpan ke file output
output_file = file_input
x.to_csv("input/{}".format(output_file), sep=';', index=False)
print(f"Data berhasil disimpan ke input/{output_file}")
print(f"Total records: {len(x)}")
print(f"Records dengan data: {len(x[x['latitude'] != ''])}")
print(f"Records tanpa data: {len(x[x['latitude'] == ''])}")

Data berhasil disimpan ke input/Flat_sekolah_terdampak.csv
Total records: 4781
Records dengan data: 4781
Records tanpa data: 0
